<a href="https://colab.research.google.com/github/jlaing100/RecipeProject/blob/main/Recipe_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')
#csv.open('/content/drive/MyDrive/Recipe/archive/RecipeNLG_dataset.csv')
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Recipe/archive/RecipeNLG_dataset.csv") # dataset borrowed from kaggle

In [ ]:
data


In [ ]:
import gensim.downloader as api
#model = api.load('word2vec-google-news-300')  # A 300-dimensional model trained on Google News.
word2vec_model = api.load("glove-wiki-gigaword-200")
len(word2vec_model.index_to_key)

In [ ]:
!pip install transformers
from transformers import pipeline
token_classifier = pipeline("ner", model="davanstrien/distilbert-base-cased_fine_tuned_food_ner", tokenizer="davanstrien/distilbert-base-cased_fine_tuned_food_ner")


## **`MAIN`**

In [ ]:
word2vec_model.most_similar("wine")

In [ ]:
import re
import sys
from transformers import pipeline
sentiment_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

most_similar = word2vec_model.most_similar
sub_cue = "I don't have"

def substitution(ingredient):


    substitutes_with_scores = most_similar(ingredient, topn=5)


    substitutes = [substitute[0] for substitute in substitutes_with_scores]

    print(f"Looking for substitutes for: {ingredient}")

    if substitutes:
        print(f"Possible substitutes for {ingredient}: {', '.join(substitutes)}")
    else:
        print(f"No substitutes found for {ingredient}")
    sys.exit()



def main():

  input_string = input("How can I help?")

  # split on 'and', 'or', and punctuation
  pattern = r'[.,!?]|\b(and|or|but)\b'

  #use the re.split() function to split the string based on the pattern
  substrings = re.split(pattern, input_string)

  #filter out any empty or whitespace-only substrings
  substrings = [substring.strip() if substring is not None else "" for substring in substrings]


  substrings = [substring for substring in substrings if substring]


  recipe_count = 0
  max_recipes = 10





  unwanted_list = []
  unwanted_ingredients = []
  for substring in substrings:
      sub_cue = "I don't have "
      result = sentiment_classifier(substring)


      ingredient = substring.split(sub_cue)[-1].strip()
      substitution(ingredient)





  for substring in substrings:
      result = sentiment_classifier(substring)
      anger_score = result[0][0]['score']
      disgust_score = result[0][1]['score']
      unwanted_ingredients = token_classifier(substring)
      user_input = substring

          #identify the ingredients in the string the user used unfavorably
      if (anger_score >= 0.5) or (disgust_score >= 0.5):
        ingredients = token_classifier(substring)

        #negative_ingredients = [entity['word'] for entity in ingredients if entity['entity'] == 'U-FOOD']

        unwanted_list = [item['word'] for item in ingredients]

        for item in unwanted_list:
          print("We'll be sure to avoid", item )


        #call substitution function for each unwanted ingredient

        #iterate over each entity in output_data
        """
        for entity in negative_ingredients:
            # Check if the 'entity' key is 'U-FOOD'
                # Append the 'word' to the food_tokens_list
            if negative_ingredients:
              unwanted_ingredients.append(negative_ingredients)
        """

  for substring in substrings:
      result = sentiment_classifier(substring)
      anger_score = result[0][0]['score']
      disgust_score = result[0][1]['score']
      joy_score = result[0][3]['score']
      neutral_score = result[0][4]['score']
      recipe_count = 0

      if (joy_score >= 0.5) or (neutral_score >= 0.5):

          #pass the substring to the token identification model
          user_input = substring

          ingredients = token_classifier(user_input)


          desired_ingredients = [entity['word'] for entity in ingredients if entity['entity'] == 'U-FOOD']
          if desired_ingredients:
            print(f"Here are some great recipes with {desired_ingredients[0]}!\n")


          matching_recipes = data[data['ingredients'].apply(lambda x: all(item in x for item in desired_ingredients) and not any(item in x for item in unwanted_list))]

          for index, row in matching_recipes.iterrows():

              matched_ner = [ingredient for ingredient in desired_ingredients if ingredient in row['NER'] and ingredient not in unwanted_list]
              #print matched recipes
              print(f"Recipe: {row['title']}")
              print("Matched Ingredients (from NER):")
              print(', '.join(matched_ner))
              print("Directions:")
              print(row['directions'])
              print("\n")

              recipe_count += 1

              if recipe_count >= max_recipes:
                 break


In [ ]:
main()

In [ ]:
#content
for index, word in enumerate(word2vec_model.index_to_key):
    if index < 10:
        print(f"word #{index}/{len(word2vec_model.index_to_key)} is {word}")

SUBSTITUTION **FUNCTION**

In [ ]:
import re

def extract_and_classify_text_after_phrase(phrase, text, token_classifier):
    pattern = re.compile(rf"{re.escape(phrase)}(.*?)\.")
    match = re.search(pattern, text)

    if match:
        result = match.group(1).strip()
        result_token = token_classifier(result)
        no_ingredients = [entity['word'] for entity in result_token if entity['entity'] == 'U-FOOD']
        return no_ingredients
    else:
        return None

text_example = "I don't have any potato and chocolate."
search_phrase = "I don't have"

result = extract_and_classify_text_after_phrase(search_phrase, text_example, token_classifier)

result

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st

def main():
    st.title("Recipe Substitution Finder")
    user_input = st.text_input("Enter the ingredient you don't have or want to replace:")
    submit_button = st.button('Find Recipes')

    if submit_button and user_input:
        substituted_recipes = substitution(user_input)
        if substituted_recipes:
            st.write("Here are some recipe suggestions:")
            for recipe in substituted_recipes:
                st.write(recipe)
        else:
            st.write("No recipes found for the given ingredient.")

if __name__ == "__main__":
    main()


2023-12-08 21:51:18.796 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py Recipe.py






  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.233.207.234:8501

